In [54]:
gcs_bucket="mickael-dm-stt"
gcs_transcripts_path="transcripts"
model="long"

In [55]:
import argparse

# [START speech_transcribe_batch_multiple_files_v2]
import re
from typing import List

from google.api_core.client_options import ClientOptions
from google.api_core.operation_async import AsyncOperation
from google.cloud import storage
from google.cloud.speech_v2 import SpeechClient
from google.cloud.speech_v2.types import cloud_speech
from google.protobuf.field_mask_pb2 import FieldMask
from google.cloud import speech_v2

async def transcribe_batch_multiple_files_v2(
    project_id: str,
    file_array: [],
    gcs_output_path: str,
    model: str,
):
#) -> AsyncOperation.operation:
    """Transcribes audio from a Google Cloud Storage URI.

    Args:
        project_id: The Google Cloud project ID.
        file_array: An array of GCS URIs to transcript with associated language code.
        gcs_output_path: The Cloud Storage URI to which to write the transcript.
        model: The STT model to use for transcription ("long" or "chirp").

    Returns:
        The BatchRecognizeResponse message.
    """
    # Instantiates a client
    if model=="chirp":
        client = speech_v2.SpeechAsyncClient(
            client_options=ClientOptions(
                api_endpoint="europe-west4-speech.googleapis.com",
            )
        )
        location="europe-west4"
    if model=="long":
        client = speech_v2.SpeechAsyncClient()
        location="global"

    default_config = cloud_speech.RecognitionConfig(
        auto_decoding_config=cloud_speech.AutoDetectDecodingConfig(),
        language_codes=["en-US"],
        model=model,
        features=cloud_speech.RecognitionFeatures(
            enable_automatic_punctuation=True,
        ),
    )
    
    files=[]
    
    for file in file_array:
        config = cloud_speech.RecognitionConfig(
            language_codes=[file["language_code"]],
        )
        files.append(
            cloud_speech.BatchRecognizeFileMetadata(
                uri=file["gcs_uri"],
                config=config,
            )
        )

    request = speech_v2.BatchRecognizeRequest(
        #recognizer=f"projects/{project_id}/locations/europe-west1/recognizers/_",
        recognizer=f"projects/{project_id}/locations/{location}/recognizers/_",
        config=default_config,
        files=files,
        recognition_output_config=cloud_speech.RecognitionOutputConfig(
            gcs_output_config=cloud_speech.GcsOutputConfig(
                uri=gcs_output_path+"/"+model+"/json",
            ),
        ),
    )
    
    # Transcribes the audio into text
    operation = client.batch_recognize(request=request,timeout=530)
    print("Waiting for operation to complete...")
    response = await (await operation).result(timeout=530)
    print(response)

# [END speech_transcribe_batch_multiple_files_v2]

In [56]:
fr_gcs_uris=!(gsutil ls gs://mickael-dm-stt/source/chirp/audio/fr-FR)
file_array=[]
for uri in fr_gcs_uris:
    file_array.append({"gcs_uri": uri,"language_code": "fr-FR"})

en_gcs_uris=!(gsutil ls gs://mickael-dm-stt/source/chirp/audio/en-US)
for uri in en_gcs_uris:
    file_array.append({"gcs_uri": uri,"language_code": "en-US"})
print(file_array)
number_of_files=len(file_array)
print(f"Number of files: {number_of_files}")

[{'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8mmzj2.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8mv7he.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3e82.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3jhn.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3qv2.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3ssu.flac', 'language_code': 'fr-FR'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/en-US/x58y7eg.flac', 'language_code': 'en-US'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/en-US/x8n0jf1.flac', 'language_code': 'en-US'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/en-US/x8n29uv.flac', 'language_code': 'en-US'}, {'gcs_uri': 'gs://mickael-dm-stt/source/chirp/audio/en-US/x8n38du.flac', 'language_code': 'en-US'}]

In [57]:
import time
import asyncio

start_time = time.time()

await asyncio.gather(
    transcribe_batch_multiple_files_v2(
        project_id="mickael-dm-stt",
        file_array=file_array[:5],
        gcs_output_path="gs://"+gcs_bucket+"/"+gcs_transcripts_path,
        model=model
    ),
    transcribe_batch_multiple_files_v2(
        project_id="mickael-dm-stt",
        file_array=file_array[5:],
        gcs_output_path="gs://"+gcs_bucket+"/"+gcs_transcripts_path,
        model=model
    )
)

duration = time.time() - start_time
print(f"Took {duration} to execute")

Waiting for operation to complete...
Waiting for operation to complete...
results {
  key: "gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8mmzj2.flac"
  value {
    uri: "gs://mickael-dm-stt/transcripts/long/json/x8mmzj2_transcript_64dd5896-0000-20ae-b311-582429a7d750.json"
  }
}
results {
  key: "gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8mv7he.flac"
  value {
    uri: "gs://mickael-dm-stt/transcripts/long/json/x8mv7he_transcript_64dd5899-0000-20ae-b311-582429a7d750.json"
  }
}
results {
  key: "gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3e82.flac"
  value {
    uri: "gs://mickael-dm-stt/transcripts/long/json/x8n3e82_transcript_64dd589c-0000-20ae-b311-582429a7d750.json"
  }
}
results {
  key: "gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3jhn.flac"
  value {
    uri: "gs://mickael-dm-stt/transcripts/long/json/x8n3jhn_transcript_64dd589f-0000-20ae-b311-582429a7d750.json"
  }
}
results {
  key: "gs://mickael-dm-stt/source/chirp/audio/fr-FR/x8n3qv2.flac"
  value {
    uri: "g

In [59]:
import json
import re
from google.cloud import storage

transcript_array=!(gsutil ls gs://{gcs_bucket}/{gcs_transcripts_path}/{model}/json)

for transcript in transcript_array:
    print(f"Operation finished. Fetching results from {transcript}...")
    output_bucket, output_object = re.match(
        r"gs://([^/]+)/(.*)", transcript
    ).group(1, 2)

    # Instantiates a Cloud Storage client
    storage_client = storage.Client()

    # Fetch results from Cloud Storage
    bucket = storage_client.bucket(output_bucket)
    blob_json = bucket.blob(output_object)
    data = json.loads(blob_json.download_as_string(client=None))
    raw_transcript=""
    
    if "results" in data:
        for alternative in data["results"]:
            if "alternatives" in alternative:
                raw_transcript+=alternative["alternatives"][0]["transcript"]

    print(f"Transcript for {transcript}: {raw_transcript}")
    
    regex=gcs_bucket+"/"+gcs_transcripts_path+"/"+model+"/json/(.*)_transcript.*"
    transcript_id=re.search(regex,transcript).group(1)
   
    uri=gcs_transcripts_path+"/"+model+"/txt/"+transcript_id+".txt"
    blob_txt = bucket.blob(gcs_transcripts_path+"/"+model+"/txt/"+transcript_id+".txt")
    blob_txt.upload_from_string(raw_transcript)

Operation finished. Fetching results from gs://mickael-dm-stt/transcripts/long/json/x58y7eg_transcript_64dd0be9-0000-2f5a-8c1a-2405887423a0.json...
Transcript for gs://mickael-dm-stt/transcripts/long/json/x58y7eg_transcript_64dd0be9-0000-2f5a-8c1a-2405887423a0.json: I'd remember getting a detention or suspended for skateboarding in the hallways my principle saying a few think you're gonna grow up and be able to skateboard through the halls of your office. You got another thing coming. I built a half pipe in my office a few years ago just so I can skateboard while I worked. My name is Casey and I said and I'm a filmmaker and a YouTuber and a business owner and a dad. We're a lot of hats. hereAs born in Southeastern Connecticut in a town called yales Ferry which is like a little kind of nothing town. I got a lot of trouble as a kid like a lot of troubling grammar school just for being bad. It wasn't a lack of respect for authority those that I disagreed with the authority as I was being 